#### Libraries

In [1]:
import pickle as pkl
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
from itertools import chain
from scipy import sparse
import scipy.sparse as ss

#### Helper Functions

#### Main Loop

In [6]:
with open('Data/test_data.pkl', 'rb') as file:
    test_graphs = pkl.load(file)

with open('Data/training_data.pkl', 'rb') as file:
    train_graphs = pkl.load(file)
    
def translateGraphs(GRAPHS):
    graphs = []
    traduction={0: 50, 1: 51, 2:52, 3:53}
    for i in tqdm(range(len(GRAPHS))):
        H = GRAPHS[i].copy()
        oldlbls = nx.get_edge_attributes(H, 'labels')
        newlbls = {e: [traduction[l[0]]] for e, l in oldlbls.items()}
        #set edge labels
        nx.set_edge_attributes(H, newlbls, name='labels')
        #overwrite
        graphs.append(H)
    return graphs 

test_graphs = translateGraphs(test_graphs)

100%|██████████| 2000/2000 [00:00<00:00, 2473.35it/s]


In [3]:
from lib import getnodelblarr, getedgelabelarr, hashtodic, assignewlabels_node, assignewlabels_edge

 def WL(graphs, h=4):
    #Number of hops
    h = 4
    N = len(graphs)# N = len(train_graphs)//1
    
    hashedgraphs = graphs#train_graphs.copy()
    hashedgraphs = hashedgraphs[0:N]
    Alphabet = {}#{f'{i}': i for i in range(49)} #{}
    currentmax = 53

    #Alphabet instances for each graph over all hops
    #Initialise lv - our list of node-labels
    #       and ev - our list of edge-labels
    lv = [[] for _ in range(len(hashedgraphs))]
    el = [[] for _ in range(len(hashedgraphs))]
    for i in range(len(hashedgraphs)):
        #get node labels, edgelabels
        v = nx.get_node_attributes(hashedgraphs[i], 'labels').values()
        e = nx.get_edge_attributes(hashedgraphs[i], 'labels').values()
        #extract from list-of-list
        v = list(chain.from_iterable(v))
        e = list(chain.from_iterable(e))
        #add []+a
        lv[i] = lv[i] + v
        el[i] = el[i] + e

    print(f'Hops = {h}')
    #For each hop
    for _ in range(h):
        #For each graph
        for i in tqdm(range(len(hashedgraphs))):
            #graph to be worked with
            Gi = hashedgraphs[i].copy()
            #simplify: get biggest connected subgraph
            Gi = Gi.subgraph(sorted(nx.connected_components(Gi), key=len, reverse=True)[0])
            
            #For each node
            #Array with columns: node, label, newlabel
            lblarri_v = getnodelblarr(Gi)
            lblarri_e = getedgelabelarr(Gi)
            #Update big alphabet, hash
            Alphabet, lblarri_v, currentmax = hashtodic(Alphabet, lblarri_v, currentmax, node=True)
            #sort and reset index
            lblarri_v = lblarri_v.sort_values(by='node').reset_index(drop=True)  
            #relabel to a different graph
            # print(lblarri_v, lblarri_e, sep='\n\n')
            Gi = assignewlabels_node(Gi, lblarri_v)

            if len(lblarri_e)!=0:
                Alphabet, lblarri_e, currentmax = hashtodic(Alphabet, lblarri_e, currentmax, node=False)
                lblarri_e = lblarri_e.sort_values(by='edge').reset_index(drop=True)
                Gi = assignewlabels_edge(Gi, lblarri_e)
                e = nx.get_edge_attributes(hashedgraphs[i], 'labels').values()
                e = list(chain.from_iterable(e))
                el[i] = el[i] + e

            #assign graph-value
            hashedgraphs[i] = Gi
            #add counts of labels, reuse previous variable
            v = nx.get_node_attributes(hashedgraphs[i], 'labels').values()
            v = list(chain.from_iterable(v))
            lv[i] = lv[i] + v
            
    return hashedgraphs, Alphabet, lv, el

In [7]:
hashedgraphs, Alphabet, lv, ev = WL(train_graphs + test_graphs)

Hops = 4


100%|██████████| 8000/8000 [02:19<00:00, 57.38it/s]


#### Generate feature vectors

In [8]:
def generateFeatureVectors(hashedgraphs, Alphabet, l):
    '''Creating matrix of feature vectors for each graph'''
    M = np.zeros((len(hashedgraphs), len(Alphabet)+100))
    for i in tqdm(range(len(hashedgraphs))):
        a = pd.value_counts(l[i])
        M[i, list(a.index)] = a.values

    sM = sparse.csr_matrix(M)
    #And save so this only needs to be saved once.
    return sM

sM = generateFeatureVectors(hashedgraphs, Alphabet, lv)

100%|██████████| 8000/8000 [00:02<00:00, 3056.18it/s]


Save Data for easier reloading later in sparse format

In [9]:
from lib import save_sparse_csr, load_sparse_csr
save_sparse_csr('Data/WL_allfeatures_e.npz', sM)
save_sparse_csr('Data/WL_trainfeatvec_e.npz', sM[0:len(train_graphs)])
save_sparse_csr('Data/WL_testfeatvec_e.npz', sM[len(train_graphs):len(train_graphs+test_graphs)])

#### Create kernel matrix

In [10]:
def kernelFromFeatureVectors(filename, sparse=True):
    sM = load_sparse_csr(filename) if sparse else np.loadtxt(filename) #'Data/WLfeaturevectors.npz'
    print(f'sM.shape = {sM.shape}')
    M = ss.csr_matrix.toarray(sM)
    K = np.dot(M, M.T)
    sK = ss.csr_matrix(K)
    return sK, M

sK, M = kernelFromFeatureVectors('Data/WL_allfeatures_e.npz', sparse=True)   
print('Kernel Matrix Determined')
save_sparse_csr('Data/WLKernel_traintest_e', sK)

# Compare how much sparse arrays save space
# from sys import getsizeof
# M = ss.csr_matrix.toarray(sM)
# print(M.shape, getsizeof(M), getsizeof(sM), sep='\n')


sM.shape = (8000, 90510)
Kernel Matrix Determined


#### Classify

Load data

In [39]:
from sklearn.model_selection import train_test_split
with open('Data/training_labels.pkl', 'rb') as file:
    labels = pkl.load(file)
from lib import load_sparse_csr
# WLData = load_sparse_csr('Data/WLKernel_traintest_e.npz')
WLData = load_sparse_csr('Data/WL\WL_allfeatures_e_h8.npz.npz')
WLData = ss.csr_matrix.toarray(WLData)

Project entire dataset (train-validate-test) to PCA

In [ ]:
import numpy as np
evals, evecs = np.linalg.eigh(WLData)
numpca = 8000
WLDataNew = WLData.dot(evecs[0:numpca].T)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
WLDataNew = ss.fit_transform(WLDataNew)

Split data

In [31]:
WLTrainValid = WLDataNew[0:len(train_graphs)] #, 0:len(train_graphs)]
X_train, X_validate, y_train, y_validate = train_test_split(WLTrainValid, labels, test_size=0.3, random_state=1)
X_test = WLDataNew[len(train_graphs): len(train_graphs+test_graphs)]

Train Linear Regression Classifier

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
lr = LogisticRegression(random_state=0, solver='newton-cholesky', verbose=False)
# Fit on training data
lr.fit(X_train, y_train)

LogisticRegression(random_state=0, solver='newton-cholesky')

Predict on Validation Data

In [35]:
#Predict on validation data
pred = lr.predict(X_validate)
fpr, tpr, thresholds = metrics.roc_curve(y_validate, pred, pos_label=1)
#AUC metric
print(f'AUC: {metrics.auc(fpr, tpr)}')

AUC: 0.7563435505993202


Predict on testing data

In [36]:
X_test.shape

(2000, 8000)

In [37]:
from lib import calculateLogits
pred_test = lr.predict_proba(X_test)
logit = calculateLogits(pred_test)

Range:
 [-290.3857713260036, 36.04365338911715]


Save to relevant file

In [38]:
from lib import saveDataToFormattedSubmissionFile
ctr = int(input('Current submission attempt:\t'))
saveDataToFormattedSubmissionFile(logit, f'Data/WL_test_pred{ctr}.csv')